In [60]:
import open3d as o3d
import numpy as np

In [61]:
def convert_point_cloud_to_numpy_array(point_cloud : o3d.geometry.PointCloud) -> np.ndarray:
    return np.asarray(point_cloud.points)

In [62]:
def convert_numpy_to_point_cloud(numpy_array : np.ndarray) -> o3d.geometry.PointCloud:
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(numpy_array)
    return pcd

In [63]:
def write_zeros_in_file(count_of_points : int, path_to_new_label_file : str):
    with open(path_to_new_label_file, 'w') as file:
        file.write('0\n' * count_of_points)

In [64]:
def select_points_from_point_cloud_by_label_id(point_cloud : o3d.geometry.PointCloud, path_to_label_file : str, label_id : int) -> o3d.geometry.PointCloud:
    labels = np.fromfile(path_to_label_file, dtype=np.uint32)
    labels = labels.reshape((-1))
    pcd_point_by_id = point_cloud.select_by_index(np.where(labels == label_id)[0])
    return pcd_point_by_id

In [65]:
def segment_plane_from_point_cloud(point_cloud : o3d.geometry.PointCloud, distance : float = 0.1) -> (o3d.geometry.PointCloud, o3d.geometry.PointCloud):
    try:
        _, inliers = point_cloud.segment_plane(distance_threshold=distance,
                                             ransac_n=3,
                                             num_iterations=5000)
    except Exception:
        return (point_cloud, point_cloud.clear())
    inlier_cloud = point_cloud.select_by_index(inliers)
    outlier_cloud = point_cloud.select_by_index(inliers, invert=True)
    return (inlier_cloud, outlier_cloud)

In [66]:
def extract_point_cloud_from_bin_file(path_to_bin_file : str) -> o3d.geometry.PointCloud:
    pcd_np = np.fromfile(path_to_bin_file, dtype=np.float32).reshape(-1, 4)[:, :3]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_np[:, :3])
    return pcd

In [67]:
def create_point_cloud_by_label_list(point_cloud : o3d.geometry.PointCloud, path_to_label_file : str, label_list : list) -> o3d.geometry.PointCloud:
    numpy_arr = np.empty((0,3), float)
    for label in label_list:
        current_point_cloud = select_points_from_point_cloud_by_label_id(point_cloud, path_to_label_file, label)
        converted_to_numpy_point_cloud = convert_point_cloud_to_numpy_array(current_point_cloud)
        numpy_arr = np.append(numpy_arr, converted_to_numpy_point_cloud, axis=0)
    return convert_numpy_to_point_cloud(numpy_arr)

In [78]:
def segment_all_planes_from_point_cloud(point_cloud : o3d.geometry.PointCloud) -> list:
    all_planes = []
    inlier_cloud, outlier_cloud = segment_plane_from_point_cloud(point_cloud, 0.3)
    while outlier_cloud.has_points():
        all_planes.append(inlier_cloud)
        inlier_cloud, outlier_cloud = segment_plane_from_point_cloud(outlier_cloud, 0.3)
    if (inlier_cloud.has_points()): all_planes.append(inlier_cloud)
    
    return all_planes

In [69]:
def get_indexes_of_points(current_dict : dict, temp_point_cloud : np.ndarray) -> list:
    index_list = []
    for point in temp_point_cloud:
        triple = (point[0], point[1], point[2])
        index_list.append(current_dict[triple])
    return index_list

In [80]:
def write_ones_to_file_by_index(index_list : list, path_to_new_label_file : str):
    file = open(path_to_new_label_file, 'r')
    list_strings = file.readlines()
    file.close()
    for index in index_list:
        list_strings[index] = "1\n"
    file = open(path_to_new_label_file, 'w')
    file.writelines(list_strings)
    file.close()

In [71]:
def create_dict_of_point_cloud(point_cloud : o3d.geometry.PointCloud) -> dict:
    result_dict = {}
    numpy_main_point_cloud = convert_point_cloud_to_numpy_array(point_cloud)
    
    for idx, point in enumerate(numpy_main_point_cloud):
        triple = (point[0], point[1], point[2])
        result_dict[triple] = idx
        
    return result_dict

In [72]:
def work_with_file(current_map : str, path_to_label_file : str, label_list : list, path_to_new_label_file : str):
    main_point_cloud = extract_point_cloud_from_bin_file(current_map)
    numpy_main_point_cloud = convert_point_cloud_to_numpy_array(main_point_cloud)
    write_zeros_in_file(len(numpy_main_point_cloud), path_to_new_label_file)
    current_dict = create_dict_of_point_cloud(main_point_cloud)
    current_point_cloud = create_point_cloud_by_label_list(main_point_cloud, path_to_label_file, PLANE_LIST)
    planes_list = segment_all_planes_from_point_cloud(current_point_cloud)
    
    for plane in planes_list:
        numpy_plane = convert_point_cloud_to_numpy_array(plane)
        index_list = get_indexes_of_points(current_dict, numpy_plane)
        write_ones_to_file_by_index(index_list, path_to_new_label_file)

In [82]:
def check_get_indexes_from_txt_file(path_to_new_label_file : str) -> list:
    result_list = []
    file = open(path_to_new_label_file, 'r')
    list_strings = file.readlines()
    file.close()
    for idx, line in enumerate(list_strings):
        if (line == "1\n"):
            result_list.append(idx)
            
    return result_list

In [73]:
PATH_TO_BIN_FOLDER = "/home/pavel/dataset/sequences/00/velodyne/"
PATH_TO_LABEL_FOLDER = "/home/pavel/Downloads/dataset/sequences/00/labels/"
PATH_TO_NEW_LABEL_BINARY_FILE = "/home/pavel/Documents/plane-segmentation-research/src/label000000.txt"

In [74]:
ROAD_LABEL = 40 # - plane
SIDEWALK_LABEL = 48 # - plane
PARKING_LABEL = 44 # - plane
OTHER_GROUND_LABEL = 49 # - not plane
BUILDING_LABEL = 50 # - plane
OTHER_STRUCTURE_LABEL = 52 # - not plane
VEGETATION_LABEL = 70 # - not plane
TRUNK_LABEL = 71 # - not plane
TERRAIN_LABEL = 72 # - plane
FENCE_LABEL = 51 # - plane

### Рабочий цикл по каждой карте:

In [75]:
current_map = PATH_TO_BIN_FOLDER + "000000.bin"
current_label_file = PATH_TO_LABEL_FOLDER + "000000.label"

In [102]:
PLANE_LIST = [ROAD_LABEL, SIDEWALK_LABEL, PARKING_LABEL, BUILDING_LABEL, TERRAIN_LABEL, FENCE_LABEL]

In [103]:
work_with_file(current_map, current_label_file, PLANE_LIST, PATH_TO_NEW_LABEL_BINARY_FILE)

### Визуально проверяю какие точки получились после "прогона" алгоритма

In [104]:
main_point_cloud = extract_point_cloud_from_bin_file(current_map)

In [105]:
lst = check_get_indexes_from_txt_file(PATH_TO_NEW_LABEL_BINARY_FILE)

In [106]:
pc = main_point_cloud.select_by_index(lst)

In [107]:
o3d.visualization.draw_geometries([pc])